<a href="https://colab.research.google.com/github/PreethaVen/Coursera_Capstone/blob/main/Segmenting_and_Clustering_Neighborhoods_in_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmenting and Clustering Neighborhoods in Toronto

Importing all the required libraries for this project

In [112]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geocoder --yes  
#!pip install geocoder
#import geocoder

from geopy.geocoders import Nominatim

import folium # map rendering library

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

Reading wiki data using pandas

In [11]:
canada_wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')


## Loading the required data into a pandas dataframe and displaying it's contents 

In [31]:
raw_df = canada_wiki[0]
raw_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Checking the number of rows and columns for dataframe (raw_df)

In [40]:
print("Shape of raw_df: ", raw_df.shape)
print("Number of rows: ", raw_df.shape[0])
print("Number of columns: ", raw_df.shape[1])
print("Display the column names: ", list(raw_df.columns))

Shape of raw_df:  (180, 3)
Number of rows:  180
Number of columns:  3
Display the column names:  ['Postal Code', 'Borough', 'Neighbourhood']


## Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

Checking for Boroughs that are "Not assigned"

In [36]:
raw_df[raw_df["Borough"] == "Not assigned"].shape


(77, 3)

Dropping the rows with a "Not assigned" Borough

In [81]:
updated_df = raw_df[raw_df["Borough"] != "Not assigned"].reset_index(drop = True)
updated_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Shape of the updated dataframe after dropping "Not assigned" Boroughs


In [82]:
print("Shape of dataframe after dropping Boroughs with a 'Not assigned' value: ", updated_df.shape)

Shape of dataframe after dropping Boroughs with a 'Not assigned' value:  (103, 3)


Checking if there are any Neighbourhoods with a "Not assigned" value

In [83]:
print("Number of rows with a 'Not assigned' Neighborhood: ", updated_df[updated_df["Neighbourhood"] == "Not assigned"].shape[0])

Number of rows with a 'Not assigned' Neighborhood:  0


## Final shape of the dataframe showing the number of rows and columns before proceeding to get the latitude and the longitude coordinates of each neighborhood. 

In [84]:
print("Shape: ", updated_df.shape)
print("Number of rows: ", updated_df.shape[0])
print("Number of columns: ", updated_df.shape[1])
print("Display column names: ", updated_df.columns.values)

Shape:  (103, 3)
Number of rows:  103
Number of columns:  3
Display column names:  ['Postal Code' 'Borough' 'Neighbourhood']


# Getting the latitude and the longitude coordinates of each neighborhood. 

Using the values from the csv file as geocoder is not working as expected

In [76]:
latlng_df = pd.read_csv("http://cocl.us/Geospatial_data")
latlng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging updated_df with latlng_df to add latitude/longitude for each neighborhood

In [85]:
updated_df = updated_df.join(latlng_df.set_index("Postal Code"), on = "Postal Code")
updated_df.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Using geopy library to get the latitude and longitude values of Toronto, Canada

In [103]:
from geopy.geocoders import Nominatim

address = "Toronto, Canada"

geolocator = Nominatim(user_agent = "trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of Toronto are {} and {}.".format(latitude, longitude))

The geographical coordinates of Toronto are 43.6534817 and -79.3839347.


#Creating a map of Toronto with neighborhoods superimposed on top

In [91]:
import folium

map_toronto = folium.Map(location = [latitude, longitude], zoom_start =10)

# add markers to map
for lat, lng, borough, neighborhood in zip(updated_df["Latitude"], updated_df['Longitude'], updated_df['Borough'], updated_df['Neighbourhood']):
  label = '{}, {}'.format(neighborhood, borough)
  label = folium.Popup(label, parse_html=False)
  folium.CircleMarker(
      location = [lat,lng],
      radius = 5,
      popup = label,
      color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
  ).add_to(map_toronto)

map_toronto  


# Segmenting and Clustering only the neighborhoods in **Downtown Toronto**

In [95]:
toronto_df = updated_df[updated_df["Borough"] == 'Downtown Toronto'].reset_index(drop = True)
toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [99]:
print("Shape of toronto_df: ", toronto_df.shape)
print("Number of rows with Downtown Torronto as Borough: ", toronto_df.shape[0])

Shape of toronto_df:  (19, 5)
Number of rows with Downtown Torronto as Borough:  19


# Creating a map of Toronto with Downtown Toronto neighborhoods superimposed on top

In [188]:
import folium

map_DowntownToronto = folium.Map(location = [latitude, longitude], zoom_start =13)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df["Latitude"], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
  label = '{}, {}'.format(neighborhood, borough)
  label = folium.Popup(label, parse_html=False)
  folium.CircleMarker(
      location = [lat,lng],
      radius = 5,
      popup = label,
      color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False
  ).add_to(map_DowntownToronto)

map_DowntownToronto  


##Utilizing the Foursquare API to explore the neighborhoods and segment them

In [109]:
CLIENT_ID = '*******' 
CLIENT_SECRET = '********' 
VERSION = '20180605' # Foursquare API version
LIMIT = 100 

# Explore Neighborhoods in Downtown Torronto

In [142]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    #print(venues_list)
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhoods', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

Code to run the above function on each neighborhood and create a new dataframe called DToronto_venues

In [143]:
DToronto_venues = getNearbyVenues(names = toronto_df["Neighbourhood"], latitudes = toronto_df["Latitude"], longitudes = toronto_df["Longitude"])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


## Checking the size of the resulting dataframe

In [144]:
print(DToronto_venues.shape)
DToronto_venues.head()

(1226, 7)


,Neighborhoods,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


# Checking how many venues were returned for each neighborhood

In [146]:
DToronto_venues.groupby("Neighborhoods").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhoods,,,,,,
Berczy Park,59,59,59,59,59,59
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


## Finding out how many unique categories can be curated from all the returned venues

In [147]:
print("There are {} unique categories.".format(DToronto_venues["Venue Category"].nunique()))

There are 207 unique categories.


# Analyzing Each Neighborhood

In [148]:
DToronto_venues.head()

,Neighborhoods,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [151]:
#one hot encoding
DToronto_onehot = pd.get_dummies(DToronto_venues["Venue Category"], prefix="", prefix_sep="")

#add Neighborhood column back to dataframe
DToronto_onehot['Neighborhoods'] = DToronto_venues["Neighborhoods"]

#move neighborhood column to the first column
fixed_columns = [DToronto_onehot.columns[-1]] + list(DToronto_onehot.columns[: -1])
DToronto_onehot = DToronto_onehot[fixed_columns]

DToronto_onehot.head()

,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,

Looking at the shape of the new dataframe

In [152]:
DToronto_onehot.shape

(1226, 208)

##Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [153]:
DToronto_grouped = DToronto_onehot.groupby("Neighborhoods").mean().reset_index()
DToronto_grouped.head()

,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.016949,0.000000,0.0,0.0,0.0000,0.0,0.0000,0.016949,0.033898,0.0,0.0,0.0,0.016949,0.016949,0.0,0.033898,0.0,0.0,0.016949,0.000000,0.000000,0.000000,0.0,0.016949,0.0,0.000000,0.0,0.000000,0.000000,0.016949,0.016949,0.0,0.0000,0.000000,0.033898,0.0,0.0,0.0,0.016949,0.050847,0.084746,0.0,0.0,0.0,0.0,0.0,0.016949,0.000000,0.016949,0.0,0.016949,0.0,0.000000,0.0,0.000000,0.000000,0.016949,0.000000,0.000000,0.000000,0.0,0.000000,0.016949,0.0,0.000000,0.000000,0.0,0.000000,0.033898,0.000000,0.0,0.016949,0.000000,0.0,0.0,0.016949,0.016949,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.016949,0.016949,0.016949,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.016949,0.0,0.0,0.000000,0.000000,0.0,0.016949,0.00000,0.016949,0.016949,0.0,0.016949,0.0,0.000000,0.0,0.0,0.0,0.016949,0.016949,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.016949,0.0,0.0,0.0,0.016949,0.0,0.000000,0.0,0.0,0.0,0.0,0.016949,0.0,0.0,0.

Confirming the new size

In [154]:
DToronto_grouped.shape

(19, 208)

# Creating the new dataframe to display the top 10 venues for each neighborhood

In [155]:
#Writing a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [156]:
#Creating the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhoods'] = DToronto_grouped['Neighborhoods']

for ind in np.arange(DToronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DToronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Bakery,Seafood Restaurant,Beer Bar,Restaurant,Jazz Club,Clothing Store
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Boat or Ferry,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boutique
2,Central Bay Street,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Thai Restaurant,Salad Place,Burger Joint,Bubble Tea Shop,Park
3,Christie,Grocery Store,Café,Park,Candy Store,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Nightclub,Coffee Shop
4,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Fast Food Restaurant,Restaurant,Gay Bar,Men's Store,Mediterranean Restaurant,Hotel,Yoga Studio


# Cluster Neighborhoods

Running _k_-means to cluster the neighborhood into 5 clusters

In [162]:
from sklearn.cluster import KMeans

dtoronto_grouped_clustering = DToronto_grouped.drop("Neighborhoods", axis =1)

# run k-means clustering
kclusters = 5
kmeans = KMeans(n_clusters = kclusters, random_state =0).fit(dtoronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 3, 0, 4, 2, 2, 2, 2, 2, 2, 0, 0, 2, 1, 2, 2, 2, 2, 2],
      dtype=int32)

In [167]:
dtoronto_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752
9,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576


### Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [174]:
#Renaming column name to 'Neighborhoods' in dtoronto_df to ensure dataframes can be joined on the key column 'Neighborhoods'
dtoronto_df = toronto_df.copy()
dtoronto_df.rename(columns = {"Neighbourhood" : "Neighborhoods"}, inplace = True)
dtoronto_df.columns.values

array(['Postal Code', 'Borough', 'Neighborhoods', 'Latitude', 'Longitude'],
      dtype=object)

In [175]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dtoronto_merged = dtoronto_df

# merge dtoronto_merged with dtoronto_df to add latitude/longitude for each neighborhood
dtoronto_merged = dtoronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhoods'), on='Neighborhoods')

dtoronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhoods,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Pub,Theater,Dessert Shop,Brewery,Restaurant
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant,College Auditorium,Nightclub
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Hotel,Cosmetics Shop,Bubble Tea Shop,Café,Japanese Restaurant,Ramen Restaurant,Bookstore
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Gastropub,Cocktail Bar,American Restaurant,Seafood Restaurant,Cosmetics Shop,Restaurant,Italian Restaurant,Cheese Shop
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,2,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Bakery,Seafood Restaurant,Beer Bar,Restaurant,Jazz Club,Clothing Store


## Visualizing the resulting clusters

In [187]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dtoronto_merged['Latitude'], dtoronto_merged['Longitude'], dtoronto_merged['Neighborhoods'], dtoronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examining the Clusters

Cluster 1

In [179]:
dtoronto_merged.loc[dtoronto_merged["Cluster Labels"] ==0, dtoronto_merged.columns[[1] + list(range(5,dtoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Pub,Theater,Dessert Shop,Brewery,Restaurant
1,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Smoothie Shop,Sandwich Place,Burrito Place,Café,Portuguese Restaurant,College Auditorium,Nightclub
5,Downtown Toronto,0,Coffee Shop,Italian Restaurant,Café,Sandwich Place,Japanese Restaurant,Thai Restaurant,Salad Place,Burger Joint,Bubble Tea Shop,Park


Cluster 2

In [180]:
dtoronto_merged.loc[dtoronto_merged["Cluster Labels"] ==1, dtoronto_merged.columns[[1] + list(range(5,dtoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,1,Park,Playground,Trail,Department Store,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run


Cluster 3

In [181]:
dtoronto_merged.loc[dtoronto_merged["Cluster Labels"] ==2, dtoronto_merged.columns[[1] + list(range(5,dtoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Hotel,Cosmetics Shop,Bubble Tea Shop,Café,Japanese Restaurant,Ramen Restaurant,Bookstore
3,Downtown Toronto,2,Coffee Shop,Café,Gastropub,Cocktail Bar,American Restaurant,Seafood Restaurant,Cosmetics Shop,Restaurant,Italian Restaurant,Cheese Shop
4,Downtown Toronto,2,Coffee Shop,Cocktail Bar,Cheese Shop,Farmers Market,Bakery,Seafood Restaurant,Beer Bar,Restaurant,Jazz Club,Clothing Store
7,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Gym,Thai Restaurant,Deli / Bodega,American Restaurant,Sushi Restaurant
8,Downtown Toronto,2,Coffee Shop,Aquarium,Café,Hotel,Brewery,Fried Chicken Joint,Restaurant,Italian Restaurant,Scenic Lookout,Sporting Goods Shop
9,Downtown Toronto,2,Coffee Shop,Hotel,Café,Salad Place,Seafood Restaurant,Restaurant,Japanese Restaurant,Italian Restaurant,American Restaurant,Sushi Restaurant
10,Downtown Toronto,2,Coffee Shop,Restaurant,Hotel,Café,Gym,American Restaurant,Deli / Bodega,Seafood Restaurant,Italian Restaurant,Japanese Restaurant
11,Downtown Toronto,2,Café,Bar,Italian Restaurant,Japanese Restaurant,Bookstore,Bakery,Yoga Studio,Sandwich Place,Beer Bar,Beer Store
12,Downtown Toronto,2,Café,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Vegetarian / Vegan Restaurant,Caribbean Restaurant,Arts & Crafts Store,Bar,Gaming Cafe,Grocery Store
15,Downtown Toronto,2,Coffee Shop,Italian Restaurant,Café,Restaurant,Beer Bar,Seafood Restaurant,Cocktail Bar,Hotel,Japanese Restaurant,Bakery


Cluster 4

In [182]:
dtoronto_merged.loc[dtoronto_merged["Cluster Labels"] ==3, dtoronto_merged.columns[[1] + list(range(5,dtoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Airport,Boat or Ferry,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boutique


Cluster 5

In [183]:
dtoronto_merged.loc[dtoronto_merged["Cluster Labels"] ==4, dtoronto_merged.columns[[1] + list(range(5,dtoronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,4,Grocery Store,Café,Park,Candy Store,Restaurant,Italian Restaurant,Baby Store,Athletics & Sports,Nightclub,Coffee Shop


# Conclusion

I clustered the neighborhoods of Downtown Toronto, Canada into 5 different clusters based on it's venues obtained from Foursquare API. I chose only the top 10 venus from each neighborhood in Downtown Toronto for Clustering. The clusters provides the following understanding that can be exapanded for further analysis and research.

**Cluster 1:** This cluster seems to be centered aroung a college/university area with top venues relevant to it.

**Cluster 2:** This seems to be family friendly cluster with park, playground, train and dog run as it's top venues.

**Cluster 3:** A lot of cofee shops and cafes form the top most venues for this cluster.

**Cluster 4:** This cluster seems to be centered around the **airport** and it's venues are relevant to the airport or travel area.

**Cluster 5:** This is the only cluster with a Baby store and Candy store as it's top venues.
